In [ ]:
import io
import ast
import numpy as np
import pandas as pd
import os

#utils

def data_string_to_df(data_string):
    data_string_no_header='\n'.join(data_string.split('\n')[1:])
    data=io.StringIO(data_string_no_header)
    df=pd.read_csv(data,sep='\t',index_col=0)
    return df

def compute_retrodicted_fit(absolute_retr,frontier_retr,prediction_type='point'):
    assert prediction_type in ['point','interval']
    if prediction_type=='point':
        absolute_retr_sum = np.sum([np.abs(np.diff(ast.literal_eval(v))) for v in absolute_retr.values.ravel()])
        frontier_retr_sum = np.sum([np.abs(np.diff(ast.literal_eval(v))) for v in frontier_retr.values.ravel()])
    else:
        def extract_numbers_from_string(string):
             # Extract numbers from string like 'a (b,c,d)'
            main_num = int(string.split()[0])
            other_nums = [int(x) for x in string.split('(')[1].strip(')').split(',')]
            return [(main_num)] + [(other_nums)]
        # Process absolute retrodictions
        abs_str_values = [v for v in absolute_retr.values.ravel()]
        abs_nums = [extract_numbers_from_string(v) for v in abs_str_values]
        abs_min_diffs = [0 if v[0] in range(v[1][0],v[1][-1]+1) else np.min(np.abs(np.array(v[1])-v[0])) for v in abs_nums]
        absolute_retr_sum = np.sum(abs_min_diffs)

        # Process frontier retrodictions 
        front_str_values = [v for v in frontier_retr.values.ravel()]
        front_nums = [extract_numbers_from_string(v) for v in front_str_values]
        front_min_diffs = [0 if v[0] in range(v[1][0],v[1][-1]+1) else np.min(np.abs(np.array(v[1])-v[0])) for v in front_nums]
        frontier_retr_sum = np.sum(front_min_diffs)

    return absolute_retr_sum+frontier_retr_sum,absolute_retr_sum, frontier_retr_sum


In [ ]:


path="results/alloc_search"

RETR_FITS={}
DATA_STORE={}

for file_idx, filename in enumerate(os.listdir(path)):
    if not filename.endswith(".csv"): continue
    file_path = os.path.join(path,filename)
    store_obj=[]
    with open(file_path, 'r') as file:
        content = file.read().strip().split('\n\n')

        for line_idx,line_content in enumerate(content):
            if line_idx==0:
                config = line_content
                config_lines = config.split('\n')
                config_dict = {}
                for line in config_lines:
                    if ':' in line:
                        key, value = line.split(':', 1)
                        config_dict[key.strip()] = value.strip()

                config = config_dict
                mini_config = {
                    'filename':file_path,
                    'alloc': ast.literal_eval(config.get('fixed allocation'))
                }

            else: 
                data_string=line_content
                if line_idx==1:
                    absolute_retr=data_string_to_df(data_string.lstrip())
                elif line_idx==2:
                    frontier_retr=data_string_to_df(data_string.lstrip())
                elif line_idx==3:
                    absolute_pred=data_string_to_df(data_string.lstrip())
                elif line_idx==4:
                    frontier_pred=data_string_to_df(data_string.lstrip())

        total_retr_fit,_,_=compute_retrodicted_fit(absolute_retr,frontier_retr,prediction_type='interval')
        _data_store={'mini_config':mini_config,
                     'absolute_retr':absolute_retr,
                     'frontier_retr':frontier_retr,
                     'absolute_pred':absolute_pred,
                     'frontier_pred':frontier_pred}
        DATA_STORE[file_idx]=_data_store

DATA_STORE = dict(sorted(DATA_STORE.items(), key=lambda x: x[1]['mini_config']['alloc']))

for entry in DATA_STORE.values():

    print(entry['mini_config'],entry['absolute_pred'])